# `Gene`: ensembl, release-110

- https://www.ensembl.org/info/data/mysql.html
- https://www.ensembl.org/info/docs/api/core/core_schema.html

Install mysqlclient: https://pypi.org/project/mysqlclient/

In [ ]:
from bionty.entities._gene import EnsemblGene

version = "release-110"

## Human

In [ ]:
ensembl_gene = EnsemblGene(organism="human", version=version)

In [ ]:
df = ensembl_gene.download_df()

In [ ]:
df.head()

In [ ]:
# https://github.com/laminlabs/bionty/issues/533
df["description"] = df["description"].str.replace(r"\[.*?\]", "", regex=True)

In [ ]:
df.head()

In [ ]:
df.to_parquet("df_human__ensembl__release-110__Gene.parquet")

In [ ]:
df_legacy = ensembl_gene.download_legacy_ids_df(df)

In [ ]:
df_legacy.shape

In [ ]:
df_legacy.to_parquet("df-legacy_human__ensembl__release-110__Gene.parquet")

## Mouse

In [ ]:
ensembl_gene = EnsemblGene(organism="mouse", version=version)

In [ ]:
df = ensembl_gene.download_df()

In [ ]:
df.head()

In [ ]:
# https://github.com/laminlabs/bionty/issues/533
df["description"] = df["description"].str.replace(r"\[.*?\]", "", regex=True)

In [ ]:
df.to_parquet("df_mouse__ensembl__release-110__Gene.parquet")

In [ ]:
df_legacy = ensembl_gene.download_legacy_ids_df(df)

In [ ]:
df_legacy.shape

In [ ]:
df_legacy.head()

In [ ]:
df_legacy.to_parquet("df-legacy_mouse__ensembl__release-110__Gene.parquet")

## saccharomyces_cerevisiae

In [ ]:
ensembl_gene = EnsemblGene(organism="saccharomyces cerevisiae", version=version)

In [ ]:
df = ensembl_gene.download_df()

In [ ]:
# https://github.com/laminlabs/bionty/issues/533
df["description"] = df["description"].str.replace(r"\[.*?\]", "", regex=True)

In [ ]:
df.to_parquet("df_saccharomyces cerevisiae__ensembl__release-110__Gene.parquet")

In [ ]:
df_legacy = ensembl_gene.download_legacy_ids_df(df, col="stable_id")

In [ ]:
df_legacy.shape